<a href="https://colab.research.google.com/github/joyceee916/ATMS-597-SP-2020/blob/master/Project4_Joyce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

In [15]:
cd /content/drive/My Drive/ATMS 597/Project4

/content/drive/My Drive/ATMS 597/Project4


Tasks
1. Use multiple linear regression to predict weather using past Global Forecast System (GFS) forecast data. 

2. Use random forest regression to predict weather using past GFS forecast data. 

Predict these variables for the period 6 UTC to 6 UTC the following day: 
* Maximum Temperature (C)
* Minimum Temperature (C)
* Maximum Wind Speed (m/s)
* Total precipitation accumulation (mm)



Day 0 at 12 UTC: you will have a model run (that's given to you)
* can incorporate observations up to but not including 00UTC 
* at the point of 00UTC will predict from 06UTC to 06UTC the next day 

Have 2 observational files (can be used in training, can also be used for verification)
* KCMI_hourly.csv
* KCMI_daily_obs.csv
* ^^ have to sum hourly observations to get total precipitation, has daily values even though the column heading says "hourly"

use pr1h or pr6h to correct for seasonal biases? (seems optional?) 

from the GFS: 
* daily.tar.gz, prof.tar.gz, sfc.tar.gz (UNIX zip files)
* for each run they contain a csv file that has the GFS forecast data 
* sfc.tar.gz - has a lot of data if you want to play around with it (sounds optional)
* prof.tar.gz - some upper air values (includes ~5 pressure levels) (can include if you want to) (sounds optional)
* each run has a timestamp that starts at 12UTC

main challenge: to get data lined up in Pandas 

start simple: start with a few variables and do a multiple linear regression, make sure things are working right 

could generate a persistence variable, try to predict tomorrow's data 

then could slowly add in more variables as you get more confidence 

could select different columns and see if your biases get better or worse as you swap out different columns - could write a function for this? 

2019 as a validation set 

calculate one metric: the RMS error (performance metric)

this week, focus on getting data read in and organized 



NOW: CAN SUBSTITUTE BICYCLE RIDERS IN WITH TEMPERATURE DATA (MODULE 4 NOTEBOOK 1): PREDICT TEMPERATURE TOMORROW 

In [38]:
pd.read_csv("KCMI_daily.csv", skiprows=8)

,Date,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in),Unnamed: 5
0,2010-01-01,12,2,15,M,NaN
1,2010-01-02,6,-5,15,0.00,NaN
2,2010-01-03,10,-6,11,0.00,NaN
3,2010-01-04,11,-2,20,0.00,NaN
4,2010-01-05,10,-1,16,M,NaN
...,...,...,...,...,...,...
3654,M = Missing,NaN,NaN,NaN,NaN,NaN
3655,NC = Not Calculated,NaN,NaN,NaN,NaN,NaN
3656,Midwestern Regional Climate Center,NaN,NaN,NaN,NaN,NaN
3657,cli-MATE: MRCC Application Tools Environment,NaN,NaN,NaN,NaN,NaN
